  **1.- Propiedades de NGC 121**

---

NGC 121 es un cúmulo globular ubicado en constelación Tucana, también llamado ESO 050-SC012, y tiene coordenadas con coordenadas y la distancia métrica al cúmulo.

Ascención recta: 00 h 26 min 58,2 s

Declinación: -71° 32' 08"

Distancia métrica: 61$\pm$3 kpc

Aquí se presenta una tabla con los enrrojecimientos del cúmulo en distintas bandas:

| Instrumento | Filtro | Longitud onda (µm) | $A_{λ}$ (mag) |
|:-----------:|:------:|:------------------:|:-------------:|
|   Landolt   |   U    |        0.35        |     0.160     |
|   Landolt   |   B    |        0.43        |     0.134     |
|   Landolt   |   V    |        0.54        |     0.101     |
|   Landolt   |   R    |        0.64        |     0.080     |
|   Landolt   |   I    |        0.80        |     0.056     |
|     ACS     | F555W  |        0.54        |     0.103     |
|     ACS     | F814W  |        0.80        |     0.056     |

Fuentes:
> *https://ned.ipac.caltech.edu/cgi-bin/objsearch?objname=ngc121&extend=no&hconst=73&omegam=0.27&omegav=0.73&corr_z=1&out_csys=Equatorial&out_equinox=J2000.0&obj_sort=RA+or+Longitude&of=pre_text&zv_breaker=30000.0&list_limit=5&img_stamp=YES#NR_Distances_0*

> *https://www.nasa.gov/content/goddard/oldest-cluster/#.V6jqDNGVs8o*

> *https://es.wikipedia.org/wiki/NGC_121*

  **2.- Describiendo filtros y sistemas y extracción y grabado de las extensiones**

---

Las imágnes que se trabajaran en esta tarea se obtuvieron con ACS que es un instrumentos de tercera generacion que está instalado en el telescopio espacial Hubble y posee tres canales, Wide Field Camera (WFC), High-Resolution Camera (HRC) y Solar Blind Camera (SBC).

HRC presenta la siguientes características:

|   Características     |         Medidas          |
|:---------------------:|:------------------------:|
|    Campo de Vista     |   29.1x26.1 arcseconds   |
|  Tamaño del Detector  |        1024x1024         |
|   Longitud de Onda    |       200-1050 nm        |

Los filtros utilizados se describen a continuación y el filtro mas parecido en el sistema Johnson:

| Instrumento | Filtro | Longitud onda (µm) |
|:-----------:|:------:|:------------------:|
|   Johnson   |   V    |        0.55        |
|   Johnson   |   I    |        0.81        |
|     ACS     | F555W  |        0.54        |
|     ACS     | F814W  |        0.80        |

Los archivos presentan 5 extensiones:

| Nro |  Nombre  |      Tipe       |           Descripción              |
|:---:|:--------:|:---------------:|:----------------------------------:|
|  0  |  PRIMARY |   PrimaryHDU    | Presenta el header de las imágenes |
|  1  |   SCI    |    ImageHDU     |  Contiene la imágenes de ciencia   |
|  2  |   WHT    |    ImageHDU     | Contiene el inverso de la varianza |
|  3  |   CTX    |    ImageHDU     |                                    |
|  4  |  HDRTAB  |   BinTableHDU   |                                    |

Para extraer y grabar las extensiones que contienen las imágenes y los inversos de las varianzas se procede:

In [ ]:
from astropy.io import fits
hdulist = fits.open('hst_10396_a4_acs_hrc_f555w_drz.fits')
hdulist.info()
fits.writeto("sci_{}".format("f555w_NRI.fits"), hdulist[1].data, hdulist[1].header, clobber=True)
fits.writeto("invar_{}".format("f555w_NRI.fits"), hdulist[2].data, hdulist[2].header, clobber=True)
hdulist.close()
hdulist = fits.open('hst_10396_a4_acs_hrc_f814w_drz.fits')
hdulist.info()
fits.writeto("sci_{}".format("f814w_NRI.fits"), hdulist[1].data, hdulist[1].header, clobber=True)
fits.writeto("invar_{}".format("f814w_NRI.fits"), hdulist[2].data, hdulist[2].header, clobber=True)
hdulist.close()

Fuentes:
> *http://www.stsci.edu/hst/acs*

> *https://www.spacetelescope.org/about/general/instruments/acs/*

> *http://www.das.uchile.cl/~mhamuy/courses/AS750/tema19.html*

  **3.- Detección de fuentes y medición de fotometría**

---

Para detectar la fuentes se determinan ciertos los parámetros en el archivo *default.sex*. Para darle pesos a las cuentas de los pixeles configuramos *WEIGHT_TYPE* en *MAP_WEIGH* y le damos la imágenes de los inverzos de las varianzas *invar_filtro_NRI.fits* a *WEIGHT_IMAGE*. De la literatura (fuente indicada más abajo) se obtiene que el *GAIN* y el *SEEING_FWHM* con valores de 2 y 0.065 respectivamente. *PIXEL_SCALE* se configura en 0 para que el programa lo lea directamente del header. Con DS9 se observan las imágenes y se determina que hay fuentes con 6 pixeles, por lo que se configura *DETECT_MINAREA* en.*PHOT_APERTURES* también se iguala a 6. Finalmente *MAG_ZEROPOINT* se determina en 25.

Para el archivo de salida, se obtienen las cantidades que se descomentan en el archivo *default.param*. Dichas cantidades son *MAG_APER*, *MAGERR_APER*, *ALPHA_J2000* y *DELTA_J2000*.

Se hace correr *SExtractor* con el comando *sextractor nombreimagen.fits -c default.sex* para detectar las fuentes.
Se detectaron 5722 fuentes para el filtro f555w y 4735 fuentes para el filtro f814w.

Fuente:
> *http://www.stsci.edu/hst/acs/documents/handbooks/cycle19/c05_imaging7.html*

  **4.- Detección de fuentes, medición de fotometría y creación de diagrama color magnitud**

---

Para hacer la unión de los catálogos hacemos:

In [6]:
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

catalogos = []
coordenadas = []

f555w = ascii.read('f555w_NRI.cat')
catalogos.append(f555w)
coordenadas.append(SkyCoord(ra=f555w['ALPHA_J2000'],dec=f555w['DELTA_J2000'],unit='deg'))

f814w = ascii.read('f814w_NRI.cat')
catalogos.append(f814w)
coordenadas.append(SkyCoord(ra=f814w['ALPHA_J2000'],dec=f814w['DELTA_J2000'],unit='deg'))

idx, sep2d, dist3d = SkyCoord.match_to_catalog_sky(coordenadas[0],coordenadas[1])
union = list(zip(range(0, len(idx)), idx, sep2d))

par1, par2, par3 = zip(*union)
catalogos[0] = catalogos[0][list(par1)]
catalogos[1] = catalogos[1][list(par2)]

Lf555w = list(map(lambda x: -2.5*np.log10(x), catalogos[0]['MAG_APER']))
Lf814w = list(map(lambda x: -2.5*np.log10(x), catalogos[1]['MAG_APER']))

El sistema VEGAMAG define un punto cero de acuerdo a la distribución espectral de la estrella Vega de tipo A.

Los elementos a considerar para corregir la magnitud son:

El zero point instrumental es la magnitud de una fuente que produce 1 cuenta por segundo ($m_0$).

La corrección de apertura es la corrección a la magnitud de una fuente que se le aplica debido a que tan angosta o ancha es la apertura de recoleccion de fotones ($CA$).

El enrojecimiento es una medida de cuanto baja el flujo a cortas logitudes de onda (normalmente en el azul) con respecto a longitudes de onda mas largas (rojo) ($A_{λ}$).

Las calibraciones de las magnitudes en el sistema $m_{λ} = -2.5\log_{10}(F)+m_0-A_λ-CA$

Los valores para las correcciones en cada filtro son las siguientes:

f555w: $m_{0}$=25.26, CA=0.24, $A_{V}$=0.1

f814w: $m_{0}$=24.85, CA=0.43, $A_{V}$=0.06

Hacemos el diagrama color-magnitud con el siguiente código:

In [7]:
import matplotlib.pyplot as plt

Lf555w = list(map(lambda x: x+25.255-0.241-0.103,Lf555w))
Lf814w = list(map(lambda x: x+24.849-0.425-0.056,Lf814w))

axis_x = list(map(lambda x: x[0]-x[1],zip(Lf555w,Lf814w)))
axis_y = Lf555w

plt.scatter(axis_x,axis_y,marker='o',c='Black',s=1)
plt.xlabel('F555W - F814W')
plt.ylabel('F555W')
plt.gca().invert_xaxis()
plt.tight_layout()
plt.show()

Fuente:
> *http://www.stsci.edu/documents/dhb/web/c32_wfpc2dataanal.fm1.html*

> *https://es.wikipedia.org/wiki/Enrojecimiento_interestelar*

> *http://ned.ipac.caltech.edu/cgi-bin/objsearch?search_type=Obj_id&objid=3284737&objname=1&img_stamp=YES&hconst=73.0&omegam=0.27&omegav=0.73&corr_z=1*